# A08 - Bootstrapping

In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, RidgeCV

## Advertising

In [36]:
data = pd.read_csv("Advertising.csv")

In [37]:
X = data[['TV', 'radio', 'newspaper']].values
y = data['sales'].values

In [38]:
model = LinearRegression().fit(X, y)

In [39]:
model.intercept_

np.float64(2.9388893694594085)

In [40]:
model.coef_[0]

np.float64(0.04576464545539761)

In [41]:
model.coef_[1]

np.float64(0.18853001691820456)

In [42]:
model.coef_[2]

np.float64(-0.00103749304247632)

In [43]:
B = 1000
n = len(data)
boot_betas = []

In [44]:
for i in range(B):
    idx = np.random.randint(0, n, n)
    m = LinearRegression().fit(X[idx], y[idx])
    boot_betas.append(np.r_[m.intercept_, m.coef_])

In [45]:
boot_betas = np.array(boot_betas)
beta_means = boot_betas.mean(axis=0)
beta_se = boot_betas.std(axis=0, ddof=1)

In [46]:
beta_means

array([ 2.94636343e+00,  4.56965825e-02,  1.88325688e-01, -7.44393666e-04])

In [47]:
beta_se

array([0.33653976, 0.00194385, 0.01078368, 0.0063903 ])

### Ridge

In [48]:
alphas = np.logspace(-3, 3, 50)
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv.fit(X, y)
alpha_opt = ridge_cv.alpha_
alpha_opt

C:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


np.float64(138.9495494373136)

In [49]:
ridge_best = Ridge(alpha=alpha_opt)
ridge_best.fit(X, y)
ridge_best.intercept_, ridge_best.coef_

(np.float64(2.9497076475063047),
 array([ 0.04576438,  0.18784897, -0.00087175]))

In [50]:
B = 1000
n = len(data)
boot_betas = []
for i in range(B):
    idx = np.random.randint(0, n, n)
    m = Ridge(alpha=alpha_opt).fit(X[idx], y[idx])
    boot_betas.append(np.r_[m.intercept_, m.coef_])    
boot_betas = np.array(boot_betas)

In [51]:
beta_means = boot_betas.mean(axis=0)
beta_se = boot_betas.std(axis=0, ddof=1)

In [52]:
beta_means

array([ 2.95533927e+00,  4.57832817e-02,  1.87879776e-01, -8.58523595e-04])

In [53]:
beta_se

array([0.33437127, 0.00191411, 0.01086211, 0.00633948])

In [88]:
import pandas as pd
import numpy as np

tabla = pd.DataFrame(
    {
        "OLS original": [
            2.9388893694594085,
            0.04576464545539761,
            0.18853001691820456,
            -0.00103749304247632
        ],
        "OLS bootstrap mean": [
            2.94636343,
            0.0456965825,
            0.188325688,
            -0.000744396666
        ],
        "OLS bootstrap SE": [
            0.33653976,
            0.00194385,
            0.01078368,
            0.0063903
        ],
        "Ridge (alpha óptimo)": [
            2.949786745630374,
            0.04576438,
            0.18784897,
            -0.000887175
        ],
        "Ridge bootstrap mean": [
            2.95533972,
            0.0457832817,
            0.187879776,
            -0.000858523595
        ],
        "Ridge bootstrap SE": [
            0.33437127,
            0.00191411,
            0.01086211,
            0.006339481
        ]
    },
    index=["β0 (intercepto)", "β1 (TV)", "β2 (radio)", "β3 (newspaper)"]
)

tabla

,OLS original,OLS bootstrap mean,OLS bootstrap SE,Ridge (alpha óptimo),Ridge bootstrap mean,Ridge bootstrap SE
β0 (intercepto),2.938889,2.946363,0.336540,2.949787,2.955340,0.334371
β1 (TV),0.045765,0.045697,0.001944,0.045764,0.045783,0.001914
β2 (radio),0.188530,0.188326,0.010784,0.187849,0.187880,0.010862
β3 (newspaper),-0.001037,-0.000744,0.006390,-0.000887,-0.000859,0.006339


### Interpretación

Los resultados muestran claramente que la variable que realmente impulsa las ventas es la publicidad en TV, porque su coeficiente es muy estable, en OLS vale 0.04576, el promedio del bootstrap queda en 0.04570, y su variación es pequeñísima (SE = 0.00194). Incluso cuando aplicamos regularización Ridge, prácticamente no cambia, 0.04576 en el modelo regularizado y 0.04578 con bootstrap, lo que quiere decir que el efecto de TV es fuerte y consistente sin importar cómo manipulemos los datos. La publicidad en radio también aporta bastante: su coeficiente OLS es 0.18853, el bootstrap lo deja casi igual en 0.18833, y aunque tiene más variación que TV (SE = 0.01078), sigue siendo muy estable. Ridge tampoco cambia mucho su efecto (0.18785 en el modelo regularizado, 0.18788 en el bootstrap), lo que confirma que radio es otro buen predictor de ventas.

En contraste, el gasto en newspaper prácticamente no tiene impacto. Su coeficiente en OLS es casi cero (–0.00104), y el bootstrap lo deja también cerca de cero (–0.00074), con más variación relativa (SE = 0.00639), lo que muestra que este predictor no aporta información real. Incluso con Ridge sigue igual de irrelevante, –0.000887 en el modelo regularizado y –0.000859 en el bootstrap, confirmando que invertir en periódico no ayuda a aumentar ventas. Finalmente, el intercepto también es bastante estable: en OLS vale 2.9389, en el bootstrap queda en 2.94636, y Ridge lo deja en 2.94979, con un error estándar muy similar en ambos métodos (SE = 0.33–0.34). En resumen, TV es el predictor más fuerte y confiable, radio también ayuda de manera clara, newspaper no sirve, y Ridge apenas modifica los resultados porque el modelo ya es muy estable desde el principio.

## Default

In [67]:
from sklearn.linear_model import LogisticRegression

In [68]:
data = pd.read_csv("Default.csv")

In [69]:
data["default"] = data["default"].astype("category")
data["student"] = data["student"].astype("category")

In [70]:
y = data["default"] == "Yes"
student_bin = (data["student"] == "Yes").astype(int)
X = np.column_stack([data["balance"], data["income"], student_bin])

In [71]:
logit = LogisticRegression(max_iter=1000).fit(X, y)

In [72]:
logit.intercept_

array([-10.90179013])

In [73]:
logit.coef_
logit.coef_
logit.coef_

array([[ 5.73059539e-03,  3.96182665e-06, -6.12573924e-01]])

In [74]:
B = 1000
n = len(data)
boot_betas = []

In [75]:
for i in range(B):
    idx = np.random.randint(0, n, n)   # remuestreo con reemplazo
    m = LogisticRegression(max_iter=1000).fit(X[idx], y[idx])
    boot_betas.append(np.r_[m.intercept_, m.coef_[0]])

In [76]:
boot_betas = np.array(boot_betas)
beta_means = boot_betas.mean(axis=0)
beta_se    = boot_betas.std(axis=0, ddof=1)

In [77]:
beta_means

array([-1.09635319e+01,  5.75545981e-03,  4.42119131e-06, -6.11722476e-01])

In [78]:
beta_se

array([5.03315238e-01, 2.33249471e-04, 8.12180086e-06, 2.30797188e-01])

### Ridge

In [79]:
alphas = np.logspace(-3, 3, 100)
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=False)
ridge_cv.fit(X, y)
alpha_opt = ridge_cv.alpha_
alpha_opt

C:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


np.float64(376.49358067924715)

In [80]:
ridge_best = Ridge(alpha=alpha_opt).fit(X, y)
ridge_best.intercept_, ridge_best.coef_

(np.float64(-0.08449458945114863),
 array([ 1.32424515e-04,  2.77474538e-07, -7.23381085e-03]))

In [81]:
B = 1000
n = len(data)
boot_betas = []
for i in range(B):
    idx = np.random.randint(0, n, n) 
    m = Ridge(alpha=alpha_opt).fit(X[idx], y[idx])
    boot_betas.append(np.r_[m.intercept_, m.coef_])
boot_betas = np.array(boot_betas)

In [83]:
beta_means = boot_betas.mean(axis=0)
beta_se = boot_betas.std(axis=0, ddof=1)

In [84]:
beta_means

array([-8.48287663e-02,  1.32583519e-04,  2.85104859e-07, -7.20232959e-03])

In [85]:
beta_se

array([7.93393180e-03, 6.87802075e-06, 1.60496619e-07, 4.15231315e-03])

In [89]:
import pandas as pd
import numpy as np

tabla_default = pd.DataFrame(
    {
        "Logistic original": [
            -10.9017,        # intercepto
            5.73059539e-03,  # balance
            3.96182656e-06,  # income
            -6.12573924e-02  # student
        ],
        "Logistic bootstrap mean": [
            -1.09635319e+01,
            5.75545981e-03,
            4.42119311e-06,
            -6.11722476e-02
        ],
        "Logistic bootstrap SE": [
            5.03315238e-01,
            2.33249471e-04,
            8.12180086e-06,
            2.30797188e-01
        ],
        "Ridge (alpha óptimo)": [
            -0.08449455894511486,
            1.32424515e-04,
            2.77474538e-07,
            -7.23381085e-03
        ],
        "Ridge bootstrap mean": [
            -8.48287663e-02,
            1.32583519e-04,
            2.85104859e-07,
            -7.20235295e-03
        ],
        "Ridge bootstrap SE": [
            7.93393180e-03,
            6.87820757e-06,
            1.60496619e-07,
            4.15213151e-03
        ]
    },
    index=["β0 (intercepto)", "β1 (balance)", "β2 (income)", "β3 (student)"]
)

tabla_default

,Logistic original,Logistic bootstrap mean,Logistic bootstrap SE,Ridge (alpha óptimo),Ridge bootstrap mean,Ridge bootstrap SE
β0 (intercepto),-10.901700,-10.963532,0.503315,-8.449456e-02,-8.482877e-02,7.933932e-03
β1 (balance),0.005731,0.005755,0.000233,1.324245e-04,1.325835e-04,6.878208e-06
β2 (income),0.000004,0.000004,0.000008,2.774745e-07,2.851049e-07,1.604966e-07
β3 (student),-0.061257,-0.061172,0.230797,-7.233811e-03,-7.202353e-03,4.152132e-03


### Interpretación

Los resultados muestran que el factor que más influye en la probabilidad de caer en default es claramente el balance de la tarjeta, ya que su coeficiente es positivo y bastante estable: en el modelo logístico original vale 0.005731, el promedio de los 1000 remuestreos es prácticamente igual (0.005755), y además su variación es muy baja (SE = 0.000233), lo que indica que el efecto es fuerte y consistente. La variable income, por otro lado, prácticamente no afecta el riesgo de default: su coeficiente es casi cero tanto en el modelo original (0.000004) como en el bootstrap (0.000004), y su error estándar es muy pequeño (0.000008), lo que confirma que no aporta información real a la predicción. En cambio, ser student sí tiene un efecto importante: el coeficiente es negativo, alrededor de –0.0613 en OLS y –0.06117 en el bootstrap, con una variación relativamente grande (SE = 0.23), lo que sugiere que estudiantes tienden a tener una probabilidad menor de default, aunque esta estimación es menos precisa.

El intercepto también muestra bastante inestabilidad, pasa de –10.90 en el modelo original a –10.96 en el bootstrap, con un error estándar considerable (SE = 0.50), lo que significa que esta parte del modelo depende bastante de la muestra. Cuando aplicamos regularización Ridge, los coeficientes cambian notablemente, especialmente el intercepto, que pasa de valores cercanos a –11 a aproximadamente –0.084, mostrando cómo la regularización reduce drásticamente la magnitud de los parámetros para evitar sobreajuste. Aun así, el signo y la dirección de los efectos principales se conservan: balance sigue siendo positivo (1.32e-04), income sigue siendo casi cero (2.77e-07), y student continúa siendo negativo (–0.007233). Los errores estándar del Ridge con bootstrap son mucho más pequeños que los de OLS, por ejemplo balance baja su variación a 6.87e-06 y student a 0.00415, lo que indica que Ridge produce un modelo más estable y menos sensible a fluctuaciones de la muestra. En resumen, balance es el predictor clave del default, income no aporta casi nada, ser student reduce el riesgo, y la regularización L2 hace que el modelo sea más estable aunque reduciendo la magnitud de los coeficientes.